# Coursera Capstone Final Report

Story :
I am going to Paris next month, So i need to find a living place for myself. I am about to complete coursera IBM Data Science Professional Certificate Specialization therefore i have to show my Data Science skill to find a good place in Paris.

This notebook will contain the code to scrape the data from internet with the help of Beautiful Soup and Requests.
<br/>
Plotting Graphs will be there to see the relations between different features of the data
<br/>
Choropleth Graph will be ploted with the help of Folium

    
Let's get started!

## Importing Libraries

In [1]:
import requests
from bs4 import BeautifulSoup
import json

import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('seaborn')

import seaborn as sns

import time
import tqdm



# Getting the data: Scraping

We will scrap the data from a website name seloger.com. 
We will scrap script tags which are in source code of the page and the parse the information with the help of BeautifulSoup.

Below is the search-url of 2-rooms apartment.


In [2]:
# This is search-url with different parameters
url = 'https://www.seloger.com/list.htm?org=advanced_search&idtt=1&idtypebien=1&cp=75&tri=initial&nb_pieces=2&naturebien=1,2,4&LISTING-LISTpg=2'

We are using custom headers for making request to pretent ourself as web browser.
We are doing so because sometimes webpages block the ip after so many requests

In [3]:
headers = {'User-Agent': '*',
'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
'Accept-Language': 'en-US,en;q=0.5',
'Accept-Encoding': 'gzip, deflate',
'Connection': 'keep-alive',
'Upgrade-Insecure-Requests': '1'}

In [4]:
s = requests.Session()
s.headers.update(headers)
s.get('http://www.seloger.com/')

<Response [200]>

Now we will iterate over different pages and we will find the script tag from the source code.
<br/>
There is a script tag which contain the details of Apartments.
<br/>
we will parse that details with help of json and retrieve the products/apartments from them and store in our apartment_data list

In [5]:
appartment_data = []
for i in tqdm.tqdm_notebook(range(2, 113)):
    url = url[:-1] + str(i)
    r = s.get(url, headers=headers)
    time.sleep(np.random.uniform(low=1, high=5)) 
    if r.status_code == 200:
        soup = BeautifulSoup(r.text, 'html.parser')
        for script_item in soup.find_all('script'):
            if 'var ava_data' in script_item.text:
                raw_json = script_item.text.split('=')[1][:-23]
        data = json.loads(raw_json)['products']
        appartment_data.append(data)

HBox(children=(IntProgress(value=0, max=111), HTML(value='')))

## Data to DataFrame

Lets create a big dataframe from the scraped data.
<br/>
lets create a function to convert list to dataframe.

In [6]:
def create_df(appartment_data):
    
    df = pd.concat([pd.DataFrame(item) for item in appartment_data])
    df = df.dropna()
    df = df.drop(['affichagetype', 'typedetransaction'], axis=1)
    df = df.drop_duplicates()
    df = df[['codeinsee', 'codepostal', 'etage', 'idagence', 'idannonce', 'idtiers', 'nb_photos',
            'position', 'prix', 'si_balcon', 'surface']]
    df = df.apply(lambda s: pd.to_numeric(s.str.replace(',', '.')))
    # filter out zero surface appartments
    df = df[~(df.surface == 0)]
    
    return df


In [13]:
df = create_df(appartment_data)
df.shape

(219, 11)

In [14]:
df=df.rename(columns={
    'codeinsee':'INSEE_code',
    'etage':'floor',
    'prix':'price',
    'si_balcon':'balcony',
})

In [15]:
df.head()

,INSEE_code,codepostal,floor,idagence,idannonce,idtiers,nb_photos,position,price,balcony,surface
0,750116,75116,0,156068,147793291,178982,8,0,1500,0,30.05
1,750116,75016,0,227641,148083277,263422,6,1,1360,1,41.00
2,750113,75013,0,109207,147585167,164701,13,2,1470,0,41.31
3,750117,75017,0,188694,147988845,244572,9,3,1580,1,47.00
4,750115,75015,0,102541,145643953,153256,8,4,1695,0,60.00
